In [ ]:
import pypan.ui as pan
import numpy as np
import matplotlib.pyplot as plt
import pickle
%matplotlib inline

A function for printing the results of a load flow analysis

In [ ]:
def print_load_flow(results):
    print('\n===== Generators =====')
    for name,data in results['generators'].items():
        if name not in ('Ptot','Qtot'):
            print(f'{name}: P = {data["P"]:7.2f} MW, Q = {data["Q"]:6.2f} MVAR, ' + 
                  f'I = {data["I"]:6.3f} kA, V = {data["V"]:6.3f} kV.')
    print(f'Total P = {results["generators"]["Ptot"]:6.2f} MW, ' + 
          f'total Q = {results["generators"]["Qtot"]:6.2f} MVAR')

    print('\n======= Buses ========')
    for name,data in results['buses'].items():
        print(f'{name}: voltage = {data["voltage"]:5.3f} pu, V = {data["Vl"]:7.3f} kV.')
        
    print('\n======= Loads ========')
    for name,data in results['loads'].items():
        if name not in ('Ptot','Qtot'):
            print(f'{name}: P = {data["P"]:7.2f} MW, Q = {data["Q"]:6.2f} MVAR, ' + 
                  f'I = {data["I"]:6.3f} kA, V = {data["V"]:8.3f} kV.')
    print(f'Total P = {results["loads"]["Ptot"]:6.2f} MW, ' +
          f'total Q = {results["loads"]["Qtot"]:6.2f} MVAR')

#### Some data about the loads and the voltage ratings of the buses

In [ ]:
LOADS = {
    'A': {'P': 125., 'Q': 50.},
    'B': {'P':  90., 'Q': 30.},
    'C': {'P': 100., 'Q': 35.}
}

VRATING = {
    'bus1': 16.5,
    'bus2': 18,
    'bus3': 13.8,
}
for bus in range(4,10):
    VRATING[f'bus{bus}'] = 230

#### Load the load flow computed by PowerFactory

In [ ]:
LF = {'PowerFactory': pickle.load(open('WSCC_9_bus_load_flow_PowerFactory.pkl','rb'))}

#### Load the WSCC netlist in pan

In [ ]:
netlist_file = 'wscc.pan'
ok,libs = pan.load_netlist(netlist_file)
if not ok:
    raise Exception('load_netlist failed.')

#### Define which variables to save in memory

In [ ]:
mem_vars = []
for bus in range(1, 10):
    if bus < 4:
        mem_vars.append(f'g{bus}:d')
        mem_vars.append(f'g{bus}:q')
    else:
        mem_vars.append(f'bus{bus}:d')
        mem_vars.append(f'bus{bus}:q')
for gen in range(1, 4):
    mem_vars.append(f'G{gen}:id')
    mem_vars.append(f'G{gen}:iq')

#### Run the load flow

In [ ]:
data = pan.DC('Lf', mem_vars=mem_vars, libs=libs, nettype=1, print='yes')
lf = {v: data[i][0] for i,v in enumerate(mem_vars)}
for bus in (1,2,3):
    for x in 'dq':
        lf[f'G{bus}:{x}'] = lf.pop(f'g{bus}:{x}')
        lf[f'bus{bus}:{x}'] = lf[f'G{bus}:{x}']

Convert the data returned by pan to the same format as saved in PowerFactory

In [ ]:
LF['pan'] = {key: {} for key in ('generators','buses','loads')}

Ptot, Qtot = 0, 0
for gen in range(1,4):
    key = f'G{gen}:'
    V = lf[key + 'd'] + 1j * lf[key + 'q']
    I = lf[key + 'id'] + 1j * lf[key + 'iq']
    S = V * I.conj()
    P = -S.real * 1e-6
    Q = -S.imag * 1e-6
    LF['pan']['generators'][f'G{gen}'] = {
        'P': P,
        'Q': Q,
        'I': abs(I) * 1e-3 / np.sqrt(3),
        'V': abs(V) * 1e-3 / np.sqrt(3),
        'Vl': abs(V) * 1e-3
    }
    Ptot += P
    Qtot += Q
LF['pan']['generators']['Ptot'], LF['pan']['generators']['Qtot'] = Ptot, Qtot

for bus in range(1,10):
    key = f'bus{bus}:'
    V = lf[key + 'd'] + 1j * lf[key + 'q']
    Vl = abs(V) * 1e-3
    LF['pan']['buses'][f'Bus {bus}'] = {
        'voltage': Vl / VRATING[f'bus{bus}'],
        'V': Vl / np.sqrt(3),
        'Vl': Vl
    }

for bus,load in zip((5,6,8),'ABC'):
    key = f'bus{bus}:'
    V = lf[key + 'd'] + 1j * lf[key + 'q']
    Vl = abs(V) * 1e-3
    LF['pan']['loads'][f'Load {load}'] = {
        'P': LOADS[load]['P'],
        'Q': LOADS[load]['Q'],
        'V': Vl / np.sqrt(3),
        'Vl': Vl
    }
    load = LF['pan']['loads'][f'Load {load}']
    load['I'] = np.sqrt(load['P'] ** 2 + load['Q'] ** 2) / load['Vl'] / np.sqrt(3)
LF['pan']['loads']['Ptot'] = np.sum([load['P'] for load in LOADS.values()])
LF['pan']['loads']['Qtot'] = np.sum([load['Q'] for load in LOADS.values()])

#### The load flow computed by pan

In [ ]:
print_load_flow(LF['pan'])

#### The load flow computed by PowerFactory

In [ ]:
print_load_flow(LF['PowerFactory'])